In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip3 install tokenizers wandb sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip3 install transformers huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Experiment')
os.chdir('TransformerBased')

In [6]:
OUTPUT_DIR = './emotion_recognition/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Aug 29 15:49:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# CFG

In [8]:
# ====================================================
# CFG
# ====================================================
class CFG:
    wandb=True
    _wandb_kernel='bluehills'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="microsoft/deberta-v3-large"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=50 # [0, 50, 100]
    epochs=10
    encoder_lr=1e-5 #2e-5
    decoder_lr=1e-5 #2e-5
    min_lr=5e-7
    eps=5e-7
    betas=(0.9, 0.999)
    batch_size=16
    fc_dropout=0.1
    target_size=1
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [9]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    import wandb
    try:
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        # wandb.login(key=secret_value_0)
        print('login to wandb')
        wandb.login()
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='Emotion Recognition', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

login to wandb


wandb: Currently logged in as: bluehills. Use `wandb login --relogin` to force relogin


# Library

In [10]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
from math import sqrt
import shutil
import string
import pickle
import random
import joblib
import itertools
import logging
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold

import torch
print(f"torch.__version__: {torch.__version__}")
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import torch.cuda.amp as amp

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoConfig, AutoModelForTokenClassification

import datasets
from datasets import list_datasets, load_dataset

import huggingface_hub
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.__version__: 1.12.1+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.21.2
env: TOKENIZERS_PARALLELISM=true


# Utils

In [11]:
# ====================================================
# Utils
# ====================================================

def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG.seed)

# Load Data

In [12]:
dataset_name = 'jakeazcona/short-text-labeled-emotion-classification'
my_dataset = load_dataset(dataset_name)

  0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
train, test = my_dataset['train'], my_dataset['test']

train = train.to_pandas()
test = test.to_pandas()

print(train.shape)
print(test.shape)

(24032, 2)
(6008, 2)


In [14]:
train.head()

,sample,label
0,i tend to be a window shopper when im alone be...,3
1,i will hopefully be able to feel less inhibite...,5
2,i feel very fond of my pinky kids,3
3,i feel like i had so much to write then got di...,0
4,i knew i was going to look at the mess and fee...,5


In [15]:
test.head()

,sample,label
0,i will not feel so alone anymore,5
1,"This is a huge stretch, but I would enjoy seei...",2
2,i know it will be no picnic and i will not fee...,5
3,i can write as many entries as humanly possibl...,0
4,Id give my left nut for another [NAME]. ...and...,2


# CV

In [16]:
!pip3 install -q iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

dfx = pd.get_dummies(train, columns=["label"]).groupby(["sample"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("label_") or c == "sample"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "sample"]
dfx_labels = dfx[labels]
dfx["fold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    print(len(trn_), len(val_))
    dfx.loc[val_, "fold"] = fold

train = train.merge(dfx[["sample", "fold"]], on="sample", how="left")
print(train.fold.value_counts())

19042 4756
19031 4767
19042 4756
19039 4759
19038 4760
3    4812
0    4807
4    4805
2    4804
1    4804
Name: fold, dtype: int64


In [17]:
if CFG.debug:
    display(train.groupby('fold').size())
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    display(train.groupby('fold').size())

In [18]:
train['text'] = '[CFG]' + train['sample'] + '[SEP]'
test['test'] = 'CFG' + test['sample'] + '[SEP]'

In [19]:
print('Describe train:', train.describe())
print('-------------------')
print('Describe test:', test.describe())

Describe train:               label          fold
count  24032.000000  24032.000000
mean       2.603196      2.000166
std        1.729178      1.414243
min        0.000000      0.000000
25%        1.000000      1.000000
50%        2.000000      2.000000
75%        4.000000      3.000000
max        5.000000      4.000000
-------------------
Describe test:              label
count  6008.000000
mean      2.600866
std       1.743349
min       0.000000
25%       1.000000
50%       2.000000
75%       4.000000
max       5.000000


In [20]:
train.head()

,sample,label,fold,text
0,i tend to be a window shopper when im alone be...,3,0,[CFG]i tend to be a window shopper when im alo...
1,i will hopefully be able to feel less inhibite...,5,3,[CFG]i will hopefully be able to feel less inh...
2,i feel very fond of my pinky kids,3,2,[CFG]i feel very fond of my pinky kids[SEP]
3,i feel like i had so much to write then got di...,0,2,[CFG]i feel like i had so much to write then g...
4,i knew i was going to look at the mess and fee...,5,2,[CFG]i knew i was going to look at the mess an...


In [21]:
test.head()

,sample,label,test
0,i will not feel so alone anymore,5,CFGi will not feel so alone anymore[SEP]
1,"This is a huge stretch, but I would enjoy seei...",2,"CFGThis is a huge stretch, but I would enjoy s..."
2,i know it will be no picnic and i will not fee...,5,CFGi know it will be no picnic and i will not ...
3,i can write as many entries as humanly possibl...,0,CFGi can write as many entries as humanly poss...
4,Id give my left nut for another [NAME]. ...and...,2,CFGId give my left nut for another [NAME]. ......


# Tokenizer

In [22]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Dataset

In [23]:
# ====================================================
# Define max_len
# ====================================================
# text_col = 'sample'
# lengths = []
# tk0 = tqdm(train[text_col].fillna("").values, total=len(train))
# for text in tk0:
#     length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
#     lengths.append(length)

def define_max_len_(text_col, df):
    max_lenghts = []
    tk0 = tqdm(df[text_col].fillna("").values, total=len(df))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        max_lenghts.append(length)
    LOGGER.info(f'{text_col} max(lengths): {max(max_lenghts)}')
    
    return max_lenghts

# set the max length with the longest sample
CFG.max_len = max(define_max_len_('text', train)) + 4

LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/24032 [00:00<?, ?it/s]

text max(lengths): 76
INFO:__main__:text max(lengths): 76
max_len: 80
INFO:__main__:max_len: 80


In [24]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs



class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.text = df['text'].values
        self.label = df['label'].values

    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.text[item])
        label = torch.tensor(self.label[item], dtype=torch.long)
        return inputs, label

# Model

In [25]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False, num_of_classes=6, memory_max_len=CFG.max_len):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(memory_max_len * self.config.hidden_size, num_of_classes)
        self._init_weights(self.fc)
        

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        

    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        batch_size = feature.shape[0]
        feature = torch.reshape(feature, (batch_size, -1))
        output = self.fc(self.fc_dropout(feature))
        return output

# Helper Function

In [26]:
from sklearn import metrics


def get_score(y_pred, y_true):
    log_loss = metrics.log_loss
    # trues = []
    # a1, a2, a3 = [1., 0., 0.], [0., 1., 0.], [0., 0., 1.]
    # for true in y_true:
    #     t = a1 if true == 0 else a2 if true == 1 else a3
    #     trues.append(t)
    score = log_loss(y_true, y_pred)
    return score

In [27]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        loss = criterion(y_preds, labels)
        loss = torch.masked_select(loss, labels != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
        loss = criterion(y_preds, labels)
        loss = torch.masked_select(loss, labels != -1).mean()
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions


def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [28]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds['label'].values


    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    # criterion = nn.BCEWithLogitsLoss(reduction="none")
    criterion = nn.CrossEntropyLoss()
    
    best_score = 1.

    for epoch in range(CFG.epochs):
        start_time = time.time()
        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)
        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        # predictions = predictions.reshape((len(valid_folds), CFG.max_len))


        #TODO scoring
        score = get_score(predictions, valid_labels)


        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

        torch.cuda.empty_cache()
        gc.collect()

    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']

    torch.cuda.empty_cache()
    gc.collect()
    
    return best_score

In [ ]:
if CFG.train:
    avg_score = 0
    total_fold = 0
    for fold in range(CFG.n_fold):
        if fold in CFG.trn_fold:
            best_score = train_loop(train, fold)
            avg_score += best_score
            total_fold += 1
    avg_score = avg_score / total_fold
    LOGGER.info(f"========== CV ==========")
    LOGGER.info(f'CV score = {avg_score}')

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

Epoch: [1][0/1201] Elapsed 0m 3s (remain 70m 56s) Loss: 5.6367(5.6367) Grad: nan  LR: 0.00000020  
Epoch: [1][100/1201] Elapsed 0m 28s (remain 5m 10s) Loss: 2.9336(4.3575) Grad: 462915.4062  LR: 0.00001000  
Epoch: [1][200/1201] Elapsed 0m 53s (remain 4m 26s) Loss: 2.5547(3.6791) Grad: 320011.3750  LR: 0.00001000  
Epoch: [1][300/1201] Elapsed 1m 18s (remain 3m 54s) Loss: 2.7031(3.2469) Grad: 385796.0938  LR: 0.00000999  
Epoch: [1][400/1201] Elapsed 1m 43s (remain 3m 26s) Loss: 0.5942(2.9120) Grad: 223659.3125  LR: 0.00000998  
Epoch: [1][500/1201] Elapsed 2m 8s (remain 2m 59s) Loss: 1.6807(2.6449) Grad: 249850.6250  LR: 0.00000996  
Epoch: [1][600/1201] Elapsed 2m 33s (remain 2m 32s) Loss: 0.6304(2.3840) Grad: 166657.5156  LR: 0.00000995  
Epoch: [1][700/1201] Elapsed 2m 58s (remain 2m 6s) Loss: 1.1230(2.1856) Grad: 245002.9219  LR: 0.00000993  
Epoch: [1][800/1201] Elapsed 3m 23s (remain 1m 41s) Loss: 1.7900(2.0341) Grad: 254419.0469  LR: 0.00000990  
Epoch: [1][900/1201] Elapsed 3m

Epoch 1 - avg_train_loss: 1.6059  avg_val_loss: 0.5174  time: 331s
INFO:__main__:Epoch 1 - avg_train_loss: 1.6059  avg_val_loss: 0.5174  time: 331s
Epoch 1 - Score: 1.1787
INFO:__main__:Epoch 1 - Score: 1.1787


EVAL: [300/301] Elapsed 0m 28s (remain 0m 0s) Loss: 1.8333(0.5174) 
Epoch: [2][0/1201] Elapsed 0m 0s (remain 15m 12s) Loss: 0.0015(0.0015) Grad: nan  LR: 0.00000977  
Epoch: [2][100/1201] Elapsed 0m 26s (remain 4m 43s) Loss: 0.6147(0.4172) Grad: 716023.3125  LR: 0.00000973  
Epoch: [2][200/1201] Elapsed 0m 51s (remain 4m 16s) Loss: 0.0645(0.4498) Grad: 305206.9062  LR: 0.00000969  
Epoch: [2][300/1201] Elapsed 1m 16s (remain 3m 50s) Loss: 0.3289(0.4556) Grad: 507084.5312  LR: 0.00000964  
Epoch: [2][400/1201] Elapsed 1m 42s (remain 3m 23s) Loss: 0.0041(0.4431) Grad: 21791.2559  LR: 0.00000959  
Epoch: [2][500/1201] Elapsed 2m 7s (remain 2m 57s) Loss: 0.5298(0.4492) Grad: 508516.2812  LR: 0.00000954  
Epoch: [2][600/1201] Elapsed 2m 32s (remain 2m 32s) Loss: 0.3733(0.4561) Grad: 539142.1250  LR: 0.00000948  
Epoch: [2][700/1201] Elapsed 2m 57s (remain 2m 6s) Loss: 0.0131(0.4593) Grad: 41897.1172  LR: 0.00000942  
Epoch: [2][800/1201] Elapsed 3m 22s (remain 1m 41s) Loss: 0.0891(0.4614) G

Epoch 2 - avg_train_loss: 0.4528  avg_val_loss: 0.4787  time: 331s
INFO:__main__:Epoch 2 - avg_train_loss: 0.4528  avg_val_loss: 0.4787  time: 331s
Epoch 2 - Score: 0.9112
INFO:__main__:Epoch 2 - Score: 0.9112
Epoch 2 - Save Best Score: 0.9112 Model
INFO:__main__:Epoch 2 - Save Best Score: 0.9112 Model


...

Epoch 9 - avg_train_loss: 0.0995  avg_val_loss: 0.6464  time: 317s
INFO:__main__:Epoch 9 - avg_train_loss: 0.0995  avg_val_loss: 0.6464  time: 317s
Epoch 9 - Score: 0.2668
INFO:__main__:Epoch 9 - Score: 0.2668


EVAL: [300/301] Elapsed 0m 27s (remain 0m 0s) Loss: 0.0000(0.6464) 
Epoch: [10][0/1201] Elapsed 0m 0s (remain 10m 39s) Loss: 0.0049(0.0049) Grad: nan  LR: 0.00000025  
Epoch: [10][100/1201] Elapsed 0m 24s (remain 4m 25s) Loss: 0.0008(0.0996) Grad: 13379.2822  LR: 0.00000021  
Epoch: [10][200/1201] Elapsed 0m 48s (remain 4m 1s) Loss: 0.1754(0.0824) Grad: 606094.6875  LR: 0.00000017  
Epoch: [10][300/1201] Elapsed 1m 13s (remain 3m 38s) Loss: 0.1836(0.0821) Grad: 676663.6250  LR: 0.00000014  
Epoch: [10][400/1201] Elapsed 1m 36s (remain 3m 13s) Loss: 0.8813(0.0848) Grad: 734666.0625  LR: 0.00000011  
Epoch: [10][500/1201] Elapsed 2m 1s (remain 2m 49s) Loss: 0.0000(0.0825) Grad: 142.2573  LR: 0.00000009  
Epoch: [10][600/1201] Elapsed 2m 25s (remain 2m 24s) Loss: 0.5664(0.0822) Grad: 638513.5000  LR: 0.00000006  
Epoch: [10][700/1201] Elapsed 2m 49s (remain 2m 0s) Loss: 0.0117(0.0795) Grad: 86113.0625  LR: 0.00000004  
Epoch: [10][800/1201] Elapsed 3m 13s (remain 1m 36s) Loss: 0.0017(0.08

Epoch 10 - avg_train_loss: 0.0824  avg_val_loss: 0.6500  time: 318s
INFO:__main__:Epoch 10 - avg_train_loss: 0.0824  avg_val_loss: 0.6500  time: 318s
Epoch 10 - Score: 0.2669
INFO:__main__:Epoch 10 - Score: 0.2669


EVAL: [300/301] Elapsed 0m 27s (remain 0m 0s) Loss: 0.0000(0.6500) 


========== CV ==========
INFO:__main__:========== CV ==========
CV score = 0.5081608755318541
INFO:__main__:CV score = 0.5081608755318541
